Install required libraries.



In [ ]:
# save
import json
import os

path_data_file = "data/v2"

# Save global_embeddings to a JSON file
def save_json(data, filename):
    os.makedirs(path_data_file, exist_ok=True)
    with open(path_data_file + filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

#load
def load_json(filename):
    with open(path_data_file + filename, "r", encoding="utf-8") as f:
        return json.load(f)

In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [19]:
from typing import Optional, List  # Add this line
from langchain_core.callbacks import CallbackManager
from langchain.llms.base import LLM
from ollama import chat, ChatResponse
from pydantic import BaseModel  # Pydantic's BaseModel for field definitions

class OllamaLLM(LLM, BaseModel):
    model_name: str
    verbose: bool = False

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if self.verbose:
            print(f"Sending prompt to {self.model_name}: {prompt}")
        response: ChatResponse = chat(
            model=self.model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response.message.content

    @property
    def _llm_type(self) -> str:
        return "ollama"

# Replace LlamaCpp with OllamaLLM
model = OllamaLLM(model_name="llama3.2", verbose=True)

In [24]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
#
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embd = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)


C:\Users\Iranian\AppData\Local\Temp\ipykernel_14180\2378217556.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embd = HuggingFaceEmbeddings(


In [14]:
import tiktoken
import matplotlib.pyplot as plt

## Helper Fuction to count the number of Tokensin each text
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [44]:
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://football360.ir/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()
docs = docs[:5]

# # LCEL w/ PydanticOutputParser (outside the primary LCEL docs)
# url = "https://react.dev/community"
# loader = RecursiveUrlLoader(
#     url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
# )
# docs_pydantic = loader.load()

# # LCEL w/ Self Query (outside the primary LCEL docs)
# url = "https://react.dev/learn"
# loader = RecursiveUrlLoader(
#     url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
# )
# docs_sq = loader.load()

# # Doc texts
# docs.extend([*docs_pydantic, *docs_sq])
docs_texts = [d.page_content for d in docs]

In [22]:
docs_texts = load_json("/docs_texts.json")
# save_json(docs_texts,"/docs_texts.json")

In [46]:
len(docs_texts)

1

In [12]:
# import json

# # Convert docs to a JSON-serializable format
# docs_data = [
#     {
#         "page_content": doc.page_content,
#         "metadata": doc.metadata  # Assuming `metadata` is a dictionary or JSON-serializable
#     }
#     for doc in docs
# ]

# Save to a JSON file
# with open(path_data_file + "/docs.json", "w", encoding="utf-8") as f:
#     json.dump(docs_data, f, ensure_ascii=False, indent=4)

# ----------------------------------------------
from langchain.schema import Document

# Load from the JSON file
docs_data = load_json("/docs.json")

# Reconstruct the docs list (convert dictionaries back to Document objects)
docs = [Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in docs_data]

# Example: Accessing content from loaded docs
print(docs[0].page_content)


فوتبال ۳۶۰حساب کاربریخلاصه بازی لیورپول ۲-۲ منچستر یونایتد| با گزارش عادل فردوسی‌پورهفته بیستم لیگ برتر انگلیس فصل ۲۰۲۴/۲۵لایو ۳۶۰ با عادل فردوسی‌پور؛ ویژه بازی استقلال و سپاهان با حضور مهدی رحمتیکاپیتان سپاهان در گفت‌وگو با «فوتبال 360»؛ نویدکیایِ فوتبال زنان هستمکاپیتان تیم زنان سپاهان اصفهان حاضر نیست در هیچ شرایطی از این تیم جدا شود و درباره دوره حضورش در جمع طلایی‌پوشان، اظهارات جالب‌توجهی در گفت‌وگو با «فوتبال 360» داشت.آخرین خبر از پروژه انتخاب جانشین گاریدو؛ یحیی تمایلی به بازگشت نداردبا وجود خبرهای منتشره، یحیی گل‌محمدی در مقطع کنونی تمایلی به حضور دوباره در تیم پرسپولیس ندارد.شایعات جنجالی علیه آینده کاری موسیمانه در استقلال؛ تهدیدی در کار نیستبرخلاف شایعات مطرح‌شده، پیتسو موسیمانه قصدی برای فسخ قراردادش ندارد و بخشی از مطالبات او پرداخت شده و بخشی دیگر از آن در حال پرداخت است.پدیده مالزیایی در تیررس غول‌های ایرانی؛ عارف آیمن، ستاره‌ای برای آیندهعملکرد درخشان وینگر مالزیایی باعث شده تا او در تیررس 3 تیم ایرانی قرار گیرد.Loading...Loading...رقابت‌ها و لیگ‌هاLoading... Loading.

In [15]:
# Doc texts concat
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Num tokens in all context: %s"
    % num_tokens_from_string(concatenated_content, "cl100k_base")
)

Num tokens in all context: 701657


In [16]:
# Doc texts split
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size_tok = 1000
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size_tok, chunk_overlap=0
)
texts_split = text_splitter.split_text(concatenated_content)
#
print(f"Number of text splits generated: {len(texts_split)}")

Number of text splits generated: 1304


In [13]:
save_json(texts_split, "/texts_split.json")

In [20]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 224  # Fixed seed for reproducibility

### --- Code from citations referenced above (added comments and docstrings) --- ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


### --- Our code below --- ###


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # Summarization
    # template = """Football 360 is a football site in Iran and follows all the news, results and transfers in the field of football in Iran and the world.

    # In the following section, I will give you a part of the data of this site and summarize it.

    # Details:
    # {context}
    # """

    template = """فوتبال 360 یک سایت فوتبالی در ایران است و تمامی اخبار و نتایج و نقل و انتقالات در حوزه فوتبال ایران و جهان رو دنبال میکند.

    در قسمت زیر یک بخشی از داده این سایت رو دراختیارت میزارم و اون روخلاصه کن.

    جزئیات:
    {context}"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

In [25]:
leaf_texts = docs_texts
results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

i:\AI_Project\manual_raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
i:\AI_Project\manual_raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
i:\AI_Project\manual_raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
i:\AI_Project\manual_raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
i:\AI_Project\manual_raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will b

--Generated 500 clusters--
Sending prompt to llama3.2: Human: فوتبال 360 یک سایت فوتبالی در ایران است و تمامی اخبار و نتایج و نقل و انتقالات در حوزه فوتبال ایران و جهان رو دنبال میکند.

    در قسمت زیر یک بخشی از داده این سایت رو دراختیارت میزارم و اون روخلاصه کن.

    جزئیات:
    فوتبال ۳۶۰حساب کاربریخلاصه بازی لیورپول ۲-۲ منچستر یونایتد| با گزارش عادل فردوسی‌پورهفته بیستم لیگ برتر انگلیس فصل ۲۰۲۴/۲۵لایو ۳۶۰ با عادل فردوسی‌پور؛ ویژه بازی استقلال و سپاهان با حضور مهدی رحمتیکاپیتان سپاهان در گفت‌وگو با «فوتبال 360»؛ نویدکیایِ فوتبال زنان هستمکاپیتان تیم زنان سپاهان اصفهان حاضر نیست در هیچ شرایطی از این تیم جدا شود و درباره دوره حضورش در جمع طلایی‌پوشان، اظهارات جالب‌توجهی در گفت‌وگو با «فوتبال 360» داشت.آخرین خبر از پروژه انتخاب جانشین گاریدو؛ یحیی تمایلی به بازگشت نداردبا وجود خبرهای منتشره، یحیی گل‌محمدی در مقطع کنونی تمایلی به حضور دوباره در تیم پرسپولیس ندارد.شایعات جنجالی علیه آینده کاری موسیمانه در استقلال؛ تهدیدی در کار نیستبرخلاف شایعات مطرح‌شده، پیتسو موسیمانه قصدی برای فسخ قرا

In [23]:
# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)
#Final Summaries extracted
print(len(all_texts))

2


In [24]:
# Now, use all_texts to build the vectorstore with Chroma
vectorstore = Chroma.from_texts(texts=all_texts, embedding=embd)
retriever = vectorstore.as_retriever()

In [25]:
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

c:\Users\Iranian\Desktop\New folder (2)\.venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate

# Define a ChatPromptTemplate
prompt = ChatPromptTemplate(
    input_variables=["context", "question"],
    metadata={
        "lc_hub_owner": "rlm",
        "lc_hub_repo": "rag-prompt",
        "lc_hub_commit_hash": "50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e"
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["context", "question"],
                template=(
                    "You are an assistant for question-answering tasks. "
                    "Use the following pieces of retrieved context to answer the question. "
                    "If you don't know the answer, just say that you don't know. "
                    "Use three sentences maximum and keep the answer concise.\n"
                    "Question: {question}\n"
                    "Context: {context}\n"
                    "Answer:"
                )
            )
        )
    ]
)

# Print the prompt
print(prompt)


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:"), additional_kwargs={})]


In [28]:
print(prompt.messages[0].prompt.template)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:


In [29]:
# Question
response =rag_chain.invoke("رتبه نخست نیم فصل اول لیگ فوتبال ایران چه تیمی است؟")
print(str(response))

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Sending prompt to llama3.2: Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: رتبه نخست نیم فصل اول لیگ فوتبال ایران چه تیمی است؟ 
Context: فوتبال ۳۶۰حساب کاربریپخش بازی لیورپول و منچستریونایتد با گزارش عادل فردوسی‌پور یک‌شنبه, ساعت 20:00لایو ۳۶۰ با عادل فردوسی‌پور؛ ویژه بازی استقلال و سپاهان با حضور مهدی رحمتیتشریح اهداف نقل‌وانتقالاتی سپاهان؛ حاج‌صفی برمی‌گردد، وحدت، ارشیا و اسدی می‌روند؟سپاهان با کم‌ترین ورودی و خروجی پا به نیمه دوم مسابقات خواهد گذاشت. آن‌ها در بیشترین حالت با 3 ورودی و 3 خروجی کارشان را در پنجره زمستانی به پایان خواهند برد.رونمایی از جدول (1+4) در پایان نیم‌فصل؛ صدرنشینی مشترک پرسپولیس و تراکتور!مدعیان اصلی قهرمانی لیگ برتر در مصاف با یکدیگر عملکرد جالب‌توجهی داشته‌اند و نمایش پرسپولیس و تراکتور نسبت به دیگر تیم‌ها بهتر بوده است.مهم‌ترین اتفاقات هفته پانزدهم؛ ترا